In [1]:
import time
import random
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import matplotlib as mpl

In [5]:
#数据预处理函数
def deal(data):         #对性别列做值替换
    data.loc[data['Sex'] == 'M',['Sex']] = 1 #对性别列做值替换
    data.loc[data['Sex'] == 'F',['Sex']] = -1
    data.loc[data['Sex'] == 'I',['Sex']] = 0
    return data
def Normaliza(df):  #归一化函数
    std_scaler = lambda x : (x-x.min())/x.std()
    df = df.apply(std_scaler)#赋回原数组
    return df
#数据分析
def check(data):
    for i in origin_feature:
        print(i,data[i].index,data[i].dtype)
    return

In [13]:
#误差大小评价函数
def ComputerError(trainX,trainY,w):
    xArr = np.array(trainX)
    yArr = np.array(trainY.values)
    num = xArr.shape[0] 
    result=np.array(np.dot(xArr,w)-yArr)
    err=(np.multiply(result,result).sum())/(2*num)
    return err
#岭回归
def RidgeRegression(xMat,yMat,lamda=0.2): #lamda是缩减系数
    xTx = xMat.T * xMat
    manom = xTx + np.eye(np.shape(xMat)[1]) * lamda
    if np.linalg.det(manom) == 0.0:
        print("矩阵为奇异矩阵,不能求逆")
        return
    w = manom.I * (xMat.T * yMat.T)
    return w
        
def ComputeRidgeReg(trainX, trainY, numTestPts=30):
    #x = np.hstack((np.ones((len(y),1)),x))
    num= np.array(trainX).shape[0]
    xMat = np.mat(trainX)
    yMat = np.mat(trainY).T
    wMat = np.zeros((numTestPts, np.shape(xMat)[1]))     #初始回归系数矩阵
    errMat = np.zeros((numTestPts, 1))
    for i in range(numTestPts):                          #改变lambda计算回归系数
        w = RidgeRegression(xMat, yMat, np.exp(i - 10))  #lambda以e的指数变化，最初是一个非常小的数     
        wMat[i, :] = w.T                                 #计算回归系数矩阵
        errMat[i]=ComputerError(trainX, trainY, w)
    return wMat,errMat

def main():
    #读取数据
    print("开始读取训练数据...")
    t0 = time.time()
    train = pd.read_csv('abalone.csv',low_memory=False)
    print('读取数据一共用时{}秒'.format(time.time() - t0))
    # 数据预处理
    print("开始进行数据预处理...")
    t0 = time.time()
    #check(train)
    train=deal(train)
    df=train.iloc[:,0:8]
    train.iloc[:,0:8]=Normaliza(df)
    print('数据预处理一共用时{}秒'.format(time.time() - t0))


    numtestPts=30
    trainX=train.iloc[:,0:8]
    trainY=train[['Rings']]
    testX=train.iloc[:,0:8] 
    print('岭回归')
    wMat,errMat = ComputeRidgeReg(trainX, trainY, numtestPts)

    print('特征值为:')
    errMin=np.min(errMat)
    print(wMat[errMat.argmin()])
    print('误差为:')
    print(errMin)

if __name__ =='__main__':
    main()

开始读取训练数据...
读取数据一共用时0.016556978225708008秒
开始进行数据预处理...
数据预处理一共用时0.021300792694091797秒
岭回归
特征值为:
[ 0.18241524  0.98051135  1.32735474  0.7698923   4.25494127 -4.79109938
 -1.3235795   0.82220076]
误差为:
2.596446296460973
